# Box Agents with Box AI

In this notebook, we will learn how to use the Enhanced Extract Agent with the Box AI API `/extract_structured` endpoint to extract information from a single file in Box. In the last example, we used a Box Metadata Template. In this example, we will define the fields we want directly inline in the API call itself. We will extract this data from a stock purchas agreement.

We will also show you how to call a Box AI Studio agent from the Box AI API `/ask` endpoint from a single file. For this, we will analyze a due dilligence document.

## Prerequisites

You must have completed the Setup notebook first. This will create all of the Box objects, folders, and files that you need, and will have created an environment file to help you get started and import all the libraries you will need.

## Workshop

The first step is to import all of the environment variables we need for this exercise.

In [1]:
import os
from dotenv import load_dotenv

load_dotenv(override=True)

BOX_CLIENT_ID=os.getenv('BOX_CLIENT_ID')
BOX_CLIENT_SECRET=os.getenv('BOX_CLIENT_SECRET')
BOX_USER_ID=os.getenv('BOX_USER_ID')
BOX_FOLDER_ID=os.getenv('EXERCISE6_FOLDER')
BOX_AGENT_ID=os.getenv('AI_AGENT_ID')

Next we will grab the BoxClient object from the Python SDK to authenticate ourselves to the API. We'll print out the current user's information to ensure we are properly authenticated.

In [2]:
from box_sdk_gen import BoxClient, CCGConfig, BoxCCGAuth

ccg_config = CCGConfig(
    client_id=BOX_CLIENT_ID,
    client_secret=BOX_CLIENT_SECRET,
    user_id=BOX_USER_ID,
)

ccg_auth = BoxCCGAuth(ccg_config)

client = BoxClient(ccg_auth)

print(f"{client.users.get_user_me()}")

<class 'box_sdk_gen.schemas.user_full.UserFull'> {'id': '19498290761', 'type': 'user', 'name': 'Scott Hurrey', 'login': 'shurrey+eplusadmin@boxdemo.com', 'created_at': '2022-05-26T10:57:52-07:00', 'modified_at': '2025-08-25T13:51:12-07:00', 'language': 'en', 'timezone': 'America/Los_Angeles', 'space_amount': 999999999999999, 'space_used': 3609201911, 'max_upload_size': 536870912000, 'status': 'active', 'job_title': '', 'phone': '', 'address': '', 'avatar_url': 'https://hurrey.app.box.com/api/avatar/large/19498290761'}


Now let's find the files in the exercise folder we'll need for this demo.

In [3]:
files = client.folders.get_folder_items(BOX_FOLDER_ID)

file_ids = {}
for file in files.entries:
    file_ids[file.name] = file.id

    print(f"File Name: {file.name},File ID: {file.id}")

File Name: Legal Due Dilligence Documents.docx,File ID: 1965333529583
File Name: STOCK PURCHASE AGREEMENT.docx,File ID: 1965327798264


Now we will create the AiAgentReference object that tells the Box AI API to use the enhanced extract agent. You will notice the id is a literal string, "enhanced_extract_agent" and the type is AI_AGENT_ID. This is a pattern you'll want to remember.

In [4]:
from box_sdk_gen import AiAgentReference, AiAgentReferenceTypeField

enhanced_extract_config = AiAgentReference(id="enhanced_extract_agent", type=AiAgentReferenceTypeField.AI_AGENT_ID)

With the AiAgentReference object defined, the next step is to reference that object in the ai_agent argument of the create_ai_extract_structured method. 

For the rest of the method, we are defining the file_ids as the file we want to interact with, and for fields, we are defining inline the fields we want to extract. You can use a Box Metadata Template if the fields are always the same. You can also define the fields inline if the Metadata Template doesn't meet your needs or the fields you extract might vary from run to run.

In [5]:
from box_sdk_gen import (
    AiItemBase,
    CreateAiExtractStructuredFields,
    CreateAiExtractStructuredFieldsOptionsField
)

ai_response = client.ai.create_ai_extract_structured(
    [AiItemBase(id=file_ids["STOCK PURCHASE AGREEMENT.docx"])],
    fields=[
        CreateAiExtractStructuredFields(
            key="parties",
            display_name="Parties",
            description="The named parties involved",
            prompt="A comma separated list of the named parties involved",
            type="string",
        ),
        CreateAiExtractStructuredFields(
            key="effectiveDate",
            display_name="Effective date",
            description="The effective date of the contract",
            prompt="The effective date of the contract",
            type="date",
        ),
        CreateAiExtractStructuredFields(
            key="purchasePrice",
            display_name="Purchase price",
            description="The purchase price stated in the contract",
            prompt="The purchase price stated in the contract",
            type="float",
        ),
        CreateAiExtractStructuredFields(
            key="summary",
            display_name="Summary",
            description="A summary of the contract in 50 words or less",
            prompt="A summary of the contract in 50 words or less including key obligations",
            type="string",
        ),
        CreateAiExtractStructuredFields(
            key="recommendation",
            display_name="Recommendation",
            description="Should we make this purchase?",
            prompt="Given the financial details, would you recommend proceeding with the purchase? Answer Yes or No.",
            type="enum",
            options=[
                CreateAiExtractStructuredFieldsOptionsField(key="Yes"),
                CreateAiExtractStructuredFieldsOptionsField(key="No"),
            ],
        ),
    ],
    ai_agent=enhanced_extract_config,
)

print(ai_response.answer)

<class 'box_sdk_gen.schemas.ai_extract_response.AiExtractResponse'> {'parties': 'Argyle LLP, Suregood Family Trust', 'effectiveDate': '2023-03-31', 'purchasePrice': 231000000, 'summary': "Argyle LLP (Buyer) agrees to purchase 51% of Erebor Life, Inc.'s common shares from Suregood Family Trust (Seller) for $231,000,000. The Seller must operate the business as usual until closing, and the Buyer must pay the purchase price at closing.", 'recommendation': 'Yes'}


Now we will use the same pattern on a different endpoint, `/ask`, and instead of calling the enhanced extract agent, we will reference an agent in Box AI Studio. These can be defined and tested in the Box admin console, or you can create them via the API.

These agents allow you to select models, tweak settings, and supply custom instructions and/or overwrite the System Prompt both holistically and at a granular level. 

First, let's run the prompt against the `ask` endpoint without the Agent.

In [9]:
from box_sdk_gen import (
    CreateAiAskMode,
    AiItemAsk,
    AiItemAskTypeField
)

prompt = """
Analyze the provided legal due diligence documents for a potential acquisition.
Summarize the key risks identified in these documents that could impact the
acquisition decision. Provide a recommendation on whether to proceed with the acquisition
based on the identified risks and a list of demands to mitigate these risks.
"""

ai_response = client.ai.create_ai_ask(
    CreateAiAskMode.SINGLE_ITEM_QA,
    prompt,
    [
        AiItemAsk(
            id=file_ids["Legal Due Dilligence Documents.docx"],
            type=AiItemAskTypeField.FILE,
        )
    ]
)

print (ai_response.answer)

The key risks identified in the legal due diligence documents for Erebor Life, Inc. include: (1) Asset Quality Verification risk related to ongoing monitoring of the $22.1 billion assets under management (AUM), which is currently under investigation by the Colorado Division of Insurance; (2) Minority Shareholder Risks stemming from Paula Wellington’s 4% ownership and her active litigation challenging strategic decisions, posing potential interference; and (3) Litigation Exposure with ongoing cases that, while not expected to materially impact valuation, require close monitoring to avoid surprises. Despite these risks, the company has a strong legal foundation, robust governance, enforceable contracts, and regulatory compliance. The recommendation is to proceed with the acquisition contingent upon implementing specific mitigation measures: (a) establish enhanced asset verification and reporting protocols to address AUM concerns; (b) negotiate clear minority shareholder protections and d

You'll notice we got a pretty good response based on our prompt. Now let's add in our agent. The way we define the agent is exactly how we defined the enhanced extract agent, but instead of the string literal for the ID, we are providing the agent's unique ID.

This agent uses a custom instruction to define the agent's focus and area of expertise, as well as selecting the Gemini 2.5 Pro reasoning model for short documents. 

In [7]:
ai_agent_config = AiAgentReference(id=BOX_AGENT_ID, type=AiAgentReferenceTypeField.AI_AGENT_ID)

With the agent defined, let's run the exact same prompt on the exact same document and compare the results.

In [10]:
ai_response = client.ai.create_ai_ask(
    CreateAiAskMode.SINGLE_ITEM_QA,
    prompt,
    [
        AiItemAsk(
            id=file_ids["Legal Due Dilligence Documents.docx"],
            type=AiItemAskTypeField.FILE,
        )
    ],
    ai_agent=ai_agent_config,
)

print (ai_response.answer)

Based on my analysis of the provided due diligence documentation for the acquisition of Erebor Life, Inc., here are my findings.

### **Summary of Key Risks**

The due diligence documents reveal several critical risks that could materially impact the proposed acquisition. The information provided is dated March 31, 2023, which, when compared to the current date of August 26, 2025, exposes significant information gaps and unresolved issues.

1.  **Critical Compliance Failure:** The company's Colorado Insurance License (No. CO-INS-4587) is listed as expiring on April 10, 2024. As of the current date, this license is expired. Operating without a valid insurance license is illegal and represents an existential threat to the business, potentially invalidating all ongoing operations and policies written since its expiration.
2.  **Unresolved Material Litigation:** Two significant lawsuits were pending as of the report date with expected resolutions in mid-2023.
    *   **Regulatory Action:**

As you can see, we got much better results. They are formatted the way we wanted and the results are much more thorough.

As you can see, the Box AI API can be a powerful tool in your developer toolbox. We offer ways to get very specific results using exactly the configuration and model you want.

To get you started, we've provided full, runnable files for these exercises. Running the following cells will generate the files for you in the exercise folders. Use them as-is, or use them as inspiration or a starting point for your workflows and applications.

In [11]:
%%writefile box_ai_enhanced_extract.py
import os
from dotenv import load_dotenv

from box_sdk_gen import (
    BoxClient,
    CCGConfig,
    BoxCCGAuth,
    AiAgentReference,
    AiAgentReferenceTypeField,
    AiItemBase,
    CreateAiExtractStructuredFields,
    CreateAiExtractStructuredFieldsOptionsField
)

load_dotenv(override=True)

BOX_CLIENT_ID=os.getenv('BOX_CLIENT_ID')
BOX_CLIENT_SECRET=os.getenv('BOX_CLIENT_SECRET')
BOX_USER_ID=os.getenv('BOX_USER_ID')
BOX_FOLDER_ID=os.getenv('EXERCISE6_FOLDER')

def get_box_client():
    ccg_config = CCGConfig(
        client_id=BOX_CLIENT_ID,
        client_secret=BOX_CLIENT_SECRET,
        user_id=BOX_USER_ID,
    )

    ccg_auth = BoxCCGAuth(ccg_config)

    client = BoxClient(ccg_auth)

    return client

def get_file_ids(client, folder_id):
    files = client.folders.get_folder_items(folder_id)

    file_ids = {}
    for file in files.entries:
        file_ids[file.name] = file.id

    return file_ids

def configure_agent():
    return AiAgentReference(id="enhanced_extract_agent", type=AiAgentReferenceTypeField.AI_AGENT_ID)

def get_fields():
    return [
        CreateAiExtractStructuredFields(
            key="parties",
            display_name="Parties",
            description="The named parties involved",
            prompt="A comma separated list of the named parties involved",
            type="string",
        ),
        CreateAiExtractStructuredFields(
            key="effectiveDate",
            display_name="Effective date",
            description="The effective date of the contract",
            prompt="The effective date of the contract",
            type="date",
        ),
        CreateAiExtractStructuredFields(
            key="purchasePrice",
            display_name="Purchase price",
            description="The purchase price stated in the contract",
            prompt="The purchase price stated in the contract",
            type="float",
        ),
        CreateAiExtractStructuredFields(
            key="summary",
            display_name="Summary",
            description="A summary of the contract in 50 words or less",
            prompt="A summary of the contract in 50 words or less including key obligations",
            type="string",
        ),
        CreateAiExtractStructuredFields(
            key="recommendation",
            display_name="Recommendation",
            description="Should we make this purchase?",
            prompt="Given the financial details, would you recommend proceeding with the purchase? Answer Yes or No.",
            type="enum",
            options=[
                CreateAiExtractStructuredFieldsOptionsField(key="Yes"),
                CreateAiExtractStructuredFieldsOptionsField(key="No"),
            ],
        ),
    ]

def chat_with_ai():
    client = get_box_client()
    file_ids = get_file_ids(client, BOX_FOLDER_ID)
    enhanced_extract_config = configure_agent()

    ai_response = client.ai.create_ai_extract_structured(
        [AiItemBase(id=file_ids["STOCK PURCHASE AGREEMENT.docx"])],
        fields=get_fields(),
        ai_agent=enhanced_extract_config,
    )

    print(ai_response.answer)

if __name__ == "__main__":  
    chat_with_ai()

Writing box_ai_enhanced_extract.py


In [13]:
%%writefile box_ai_studio_agent.py
import os
from dotenv import load_dotenv

from box_sdk_gen import (
    BoxClient,
    CCGConfig,
    BoxCCGAuth,
    AiAgentReference,
    AiAgentReferenceTypeField,
    CreateAiAskMode,
    AiItemAsk,
    AiItemAskTypeField
)

load_dotenv(override=True)

BOX_CLIENT_ID=os.getenv('BOX_CLIENT_ID')
BOX_CLIENT_SECRET=os.getenv('BOX_CLIENT_SECRET')
BOX_USER_ID=os.getenv('BOX_USER_ID')
BOX_FOLDER_ID=os.getenv('EXERCISE6_FOLDER')
BOX_AGENT_ID=os.getenv('AI_AGENT_ID')

def get_box_client():
    ccg_config = CCGConfig(
        client_id=BOX_CLIENT_ID,
        client_secret=BOX_CLIENT_SECRET,
        user_id=BOX_USER_ID,
    )

    ccg_auth = BoxCCGAuth(ccg_config)

    client = BoxClient(ccg_auth)

    return client

def get_file_ids(client, folder_id):
    files = client.folders.get_folder_items(folder_id)

    file_ids = {}
    for file in files.entries:
        file_ids[file.name] = file.id

    return file_ids

def configure_agent():
    return AiAgentReference(id=BOX_AGENT_ID, type=AiAgentReferenceTypeField.AI_AGENT_ID)

def chat_with_ai():
    client = get_box_client()
    file_ids = get_file_ids(client, BOX_FOLDER_ID)
    ai_agent_config = configure_agent()

    prompt = """
    Analyze the provided legal due diligence documents for a potential acquisition.
    Summarize the key risks identified in these documents that could impact the
    acquisition decision. Provide a recommendation on whether to proceed with the acquisition
    based on the identified risks and a list of demands to mitigate these risks.
    """

    ai_response = client.ai.create_ai_ask(
        CreateAiAskMode.SINGLE_ITEM_QA,
        prompt,
        [
            AiItemAsk(
                id=file_ids["Legal Due Dilligence Documents.docx"],
                type=AiItemAskTypeField.FILE,
            )
        ],
        ai_agent=ai_agent_config,
    )

    print (ai_response.answer)

if __name__ == "__main__":  
    chat_with_ai()

Overwriting box_ai_studio_agent.py
